# PSYN Simon Coupermant et Anthony Lwin

## 1) Importation des bibliothèques nécessaires


In [5]:
import pandas as pd  # Manipulation de données
import numpy as np  # Calculs numériques
import matplotlib.pyplot as plt  # Visualisation de données
import seaborn as sns  # Visualisation avancée

# Importation des bibliothèques de scikit-learn pour la modélisation
from sklearn.model_selection import train_test_split  # Division des données en ensembles d'entraînement et de test
from sklearn.ensemble import RandomForestClassifier  # Modèle de classification Random Forest
from sklearn.linear_model import LinearRegression  # Modèle de régression linéaire
from sklearn.pipeline import make_pipeline  # Construction de pipelines pour le prétraitement et les modèles
from sklearn.preprocessing import StandardScaler, PolynomialFeatures  # Normalisation et transformation polynomiale
from sklearn.decomposition import PCA  # Réduction de dimension avec PCA
from sklearn.metrics import accuracy_score, classification_report   # Évaluation des modèles
# Import des outils de scikit-learn nécessaires
from sklearn.model_selection import GridSearchCV  # Optimisation des hyperparamètres
from sklearn.metrics import roc_auc_score, roc_curve  # Évaluation avec AUC-ROC
from sklearn.model_selection import cross_val_score  # Validation croisée

ModuleNotFoundError: No module named 'pandas'

In [19]:
from sklearn.metrics import mean_squared_error

## 2 Acquisition des données et Exploration des données

In [20]:
# Define data path and folder
path_phone = "C:/Users/scoup/Downloads/arrow_data_psyn.csv"
df = pd.read_csv(path_phone, on_bad_lines='skip')

In [ ]:
df.shape # Dimension du jeu de données

In [ ]:
df.head() # Affichage des premières lignes

In [ ]:
print("\nInformations sur le DataFrame :")
print(df.info())

In [ ]:
print("\nStatistiques descriptives :")
print(df.describe(include='all'))

### Analyse exploratoire des données (EDA)

## 3 Préparation des données

Il y a un doublon de la valeur Maximum power dissipation(Mw), on va supprimer la deuxième colonne Maximum power dissipation car elle contient moins de données

In [ ]:
# Suppression de la colonne "Maximum power dissipation (MW"
column_to_remove = "Maximum power dissipation (MW"
if column_to_remove in df.columns:
    df = df.drop(columns=[column_to_remove])

# Vérification des colonnes restantes
print("Colonnes restantes dans le DataFrame :", df.columns.tolist())

### Analyse des données manquantes

In [ ]:
missing_data_rows = df[df.isnull().any(axis=1)]
print(f"% de données manquantes est : {(len(missing_data_rows)/len(df))*100}")

#### Analyse des données manquantes pour les différentes colonnes

In [ ]:
col = df.columns
col

In [ ]:
percentages = []
for i in range(2, 15):
   missing_col = df[df.iloc[:, i].isnull()]
   percentage = round((len(missing_col)/len(df))*100, 2)
   percentages.append(percentage)
   print(f"{col[i]} : {percentage:.2f}% de valeurs manquantes")

In [ ]:
percentages[1] # Récupération du pourcentage de valeurs manquantes pour la colonne "Type"

#### Affichage du pourcentage des données manquantes selon les différentes colonnes

In [ ]:
# Dictionnaire des pourcentages de valeurs manquantes
data = { 
   'Fabricant': percentages[0],
   'Type': percentages[1],
   'Configuration': percentages[2],
   'Nominal Zener Voltage (V)': percentages[3],
   'Zener voltage tolerance': percentages[4],
   'Maximum power dissipation (MW)': percentages[5],
   'Test Current (MA)': percentages[6],
   'Maximum reverse leaking current (UA)': percentages[7],
   'Maximum regulator current (MA)': percentages[8],
   'Maximum Zener impedance (OHM)': percentages[9],
   'Packaging': percentages[10],
   'Pin Count': percentages[11],
   'SVHC': percentages[12]
}

# Trier les données en ordre décroissant en fonction des valeurs
sorted_data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))

# Création du graphique en barres VERTICALES (bar au lieu de barh)
plt.figure(figsize=(12, 8))
plt.bar(list(sorted_data.keys()), list(sorted_data.values()))  # Utilisation de plt.bar pour vertical

# Ajout des labels et du titre
plt.ylabel('Pourcentage de valeurs manquantes')
plt.xlabel('Colonnes')
plt.title('Pourcentage de valeurs manquantes par colonne (trié décroissant)')

# Rotation des étiquettes pour une meilleure lisibilité
plt.xticks(rotation=45, ha='right')

# Ajustement automatique de la disposition
plt.tight_layout()

# Affichage du graphique
plt.show()

Les colonnes Maximum power dissipation (MW) (un des inputs) et SVHC(la colonne target) sont à étudier car ces deux colonnes sont peu remplies (respectivement 72% et 45% de données manquantes) 

### Nettoyage et Prétaitement des données

In [31]:
df_svhc = df[df['SVHC'] == 1] # Filtrage des données avec SVHC = 1 (présence de substances dangereuses)
df_Nan =  df[df['SVHC'].isna()] # Filtrage des données avec SVHC = NaN (données manquantes)
df_nvhc = df[df['SVHC'] == 0] # Filtrage des données avec SVHC = 0 (absence de substances dangereuses)

In [ ]:
df_svhc.shape # Dimension du jeu de données avec SVHC = 1  (présence de substances dangereuses)

In [ ]:
df_nvhc.shape # Dimension du jeu de données avec SVHC = 0 (absence de substances dangereuses)

In [ ]:
df_Nan.shape # Dimension du jeu de données avec SVHC = NaN (données manquantes)

In [35]:
df_train = pd.concat([df_svhc, df_nvhc]) # Concaténation des données avec SVHC = 1 et SVHC = 0

In [ ]:
df_train.head() # Affichage des premières lignes du jeu de données d'entraînement


La référence Zener étant uniquement une description de certaines caractéristiques de la diode, la colonne Réference n'est pas utile pour la prédiction

In [37]:
df_train_ss_ref = df_train.drop(columns=['Reference ZENER']) # Suppression de la colonne "Reference ZENER"

### Modification des colonnes ayant des données mixtes 

#### Pour analyser nos données et faire de la prédictions, nous avons besoin d'avoir tout nos inputs de forme numérique

Dans la colonne "Maximum regulator current (MA)" on a des valeurs de ce type 56@Ta=50C ce qui signifique que le Maximum regulator current est 56 Ma pour une température ambiante de  50°C. On doit prendre que le Ma. Mais on va supprimer la colonne "Maximum regulator current (MA)".

In [38]:
# Fonction pour extraire la valeur avant '@' ou laisser la valeur d'origine si '@' n'est pas présent
def extract_current_before_at(value):
    if isinstance(value, str) and "@" in value:
        return float(value.split('@')[0])  # Extrait la partie avant '@' comme un float
    return value  # Retourne la valeur d'origine (par ex. : 9.2)

# Fonction pour extraire la température après '@'
def extract_temperature_after_at(value):
    if isinstance(value, str) and "@" in value:
        temp_part = value.split('@')[1].strip()  # Extrait la partie après '@'
        if "=" in temp_part and "C" in temp_part:
            try:
                return float(temp_part.split('=')[1].replace('C', '').strip())  # Extrait la température entre '=' et 'C'
            except ValueError:
                return None  # Retourne None en cas d'erreur
    return None  # Retourne None si '@' n'est pas présent

In [ ]:
#test
# Cas de test
print(extract_temperature_after_at("83@Ta=50C"))  # Résultat attendu : 50
print(extract_temperature_after_at("162@Ta=25C"))  # Résultat attendu : 25
print(extract_temperature_after_at("83 @Ta=50C"))  # Résultat attendu : 50 (avec un espace avant @)
print(extract_current_before_at("50"))  # Résultat attendu : 50 (avec un espace après @)


In [40]:
# Création de la colonne pour la température ambiante avant de modifier la colonne originale
df_train_ss_ref["Maximum Regulator Current Ambient Temperature (°C)"] = df_train_ss_ref["Maximum regulator current (MA)"].apply(extract_temperature_after_at)

# Mise à jour de la colonne "Maximum regulator current (MA)" pour ne garder que les valeurs avant '@'
df_train_ss_ref["Maximum regulator current (MA)"] = df_train_ss_ref["Maximum regulator current (MA)"].apply(extract_current_before_at)

In [41]:
# Réorganiser les colonnes pour placer "Maximum Regulator Current Ambient Temperature (°C)" après "Maximum regulator current (MA)"
columns = list(df_train_ss_ref.columns)
index = columns.index("Maximum regulator current (MA)")
columns.insert(index + 1, columns.pop(columns.index("Maximum Regulator Current Ambient Temperature (°C)")))
df_train_ss_ref = df_train_ss_ref[columns]

In [ ]:
df_train_ss_ref.head()

Dans la colonne colonne Maximum Zener impedance (OHM), on va enlever les TYP sur les données. Des données de la forme 50(Typ) vont devenir 50. 

In [43]:
# extraire les valeurs avant Typ dans la colonne  Maximum Zener impedance (OHM)
def extract_current_before_typ(value):
    if isinstance(value, str) and "(Typ)" in value:
        return float(value.split('(Typ)')[0])  # Extrait la partie avant '(Typ)' comme un float
    return value  # Retourne la valeur d'origine (par ex. : 9.2)


In [ ]:
# test 
print(extract_current_before_typ("50(Typ)"))  # Résultat attendu : 50

In [ ]:
df_train_ss_ref["Maximum Zener impedance (OHM)"] = df_train_ss_ref["Maximum Zener impedance (OHM)"].apply(extract_current_before_typ)

df_train_ss_ref["Maximum Zener impedance (OHM)"].unique()

On va transformer les % de la colonne Zener voltage tolerance

In [46]:
df_train_ss_ref['Zener volatge tolerance'] = df_train_ss_ref['Zener volatge tolerance'].str.replace('%', '').astype(float) / 100

In [ ]:
df_train_ss_ref['Zener volatge tolerance'].unique()

Transformer les données catégorielles en données numériques de Maximum regulator current (MA) et de  Maximum Zener impedance (OHM)  

Tout d'abord nous allons Transformer les données catégorielles en données numériques de Maximum regulator current (MA)

In [ ]:
df_train_ss_ref['Maximum regulator current (MA)'].unique()

In [ ]:
df_train_ss_ref['Maximum regulator current (MA)'] = pd.to_numeric(
    df_train_ss_ref['Maximum regulator current (MA)'], errors='coerce'
)
df_train_ss_ref['Maximum regulator current (MA)'].unique()

Maintenant nous allons transformer les données catégorielles en données numériques de Maximum Zener impedance (OHM)

In [ ]:
df_train_ss_ref['Maximum Zener impedance (OHM)'] = pd.to_numeric(
    df_train_ss_ref['Maximum Zener impedance (OHM)'], errors='coerce'
)
df_train_ss_ref['Maximum Zener impedance (OHM)'].unique()

On va créer un nouveau csv pour vérifier que les données ont bien été traitées

In [ ]:
# Sauvegarder le fichier mis à jour
updated_file_path_2 = 'C:/Users/scoup/Downloads/updated_arrow_data_lv_with_temperature.csv'
df_train_ss_ref.to_csv(updated_file_path_2, index=False)
updated_file_path_2

In [ ]:
df_train_ss_ref.head()

In [ ]:
df_train_ss_ref["Maximum Zener impedance (OHM)"].unique()

In [ ]:
print(df_train_ss_ref.dtypes)


# PCA pour voir les principales composantes

## Peut être encoder de manière numérique les colonnes Type et Configuration  

In [ ]:
print(df_train_ss_ref.columns.tolist())

In [ ]:

# Supposons que les colonnes sont nommées "a", "b", ..., "p"
columns = list(df_train_ss_ref.columns)

# Sélection des colonnes de la 5e à l'avant-dernière, sans prendre la 3e et 4e
selected_columns = columns[1:2] + columns[4:-1]  # Exclut la 3e et 4e colonnes
print("Colonnes sélectionnées pour la PCA :", selected_columns)

# Filtrer le DataFrame pour ne garder que les colonnes sélectionnées
df_selected = df_train_ss_ref[selected_columns]

len(df_selected)

In [57]:
# Supprimer les lignes contenant des NaN dans les colonnes sélectionnées
df_selected_ss_na = df_selected.dropna()

# Standardiser les données (nécessaire pour la PCA)
df_selected_ss_na_scaled = StandardScaler().fit_transform(df_selected_ss_na)

# Effectuer la PCA
pca = PCA(n_components=2)  # Réduire à 2 dimensions principales
principalComponents = pca.fit_transform(df_selected_ss_na_scaled)

# Créer un DataFrame avec les composantes principales
principalDf = pd.DataFrame(data=principalComponents, columns=['principal component 1', 'principal component 2'])


In [ ]:
# Ajouter la colonne target pour les indices correspondants après dropna()
print(principalDf)


In [ ]:
# Obtenir les charges (importance des colonnes dans les composantes principales)
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['Principal Component 1', 'Principal Component 2'],
    index=selected_columns
)

print("Charges des colonnes dans les composantes principales :")
print(loadings)

In [ ]:
# Identifier les colonnes les plus importantes pour la première composante principale
important_columns_pc1 = loadings['Principal Component 1'].abs().sort_values(ascending=False)
print("Colonnes les plus importantes pour la première composante principale :")
print(important_columns_pc1)

In [ ]:
# Identifier les colonnes les plus importantes pour la première composante principale
important_columns_pc2 = loadings['Principal Component 2'].abs().sort_values(ascending=False)
print("Colonnes les plus importantes pour la première composante principale :")
print(important_columns_pc2)

In [ ]:
print("Variance expliquée par chaque composante :", pca.explained_variance_ratio_)
print("Variance totale expliquée :", sum(pca.explained_variance_ratio_))


### Les colonnes Fabricant, Packaging et Pin Count ne sont pas importantes car elles ont une variance de 0 dans PC1 et PC2

In [ ]:
df_selected_ss_na
len(df_selected_ss_na)

In [ ]:
# Colonnes à conserver
important_columns = [
    "Maximum power dissipation (MW)",
    "Maximum Regulator Current Ambient Temperature (°C)",
    "Test Current (MA)",
    "Maximum regulator current (MA)",
    "Maximum reverse leaking current (UA)",
    "0minal Zener Voltage (V)",
    "Maximum Zener impedance (OHM)"
]

# Filtrer le DataFrame et créer une copie explicite
df_final = df_train_ss_ref[important_columns].copy()
#Ajout de la colonne SVHC target 
df_final['SVHC'] = df_train_ss_ref['SVHC']
# Vérification des colonnes restantes
print("Colonnes restantes après réduction :", df_final.columns.tolist())

In [ ]:
df_final

# Random Forest 


In [66]:


# Supposons que 'SVHC' soit la variable cible
X = df_final[important_columns]
y = df_final['SVHC']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest simple 

In [ ]:
# Créer le modèle de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
rf_model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

# Amélioration du random Forest

In [ ]:
# Étape 1 : Création du modèle de Random Forest (avec hyperparamètres de base)
rf_model = RandomForestClassifier(random_state=42)

# Étape 2 : Validation croisée pour évaluer la robustesse du modèle
cv_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Validation croisée (Accuracy moyenne) : {np.mean(cv_scores):.4f}")

# Étape 3 : Optimisation des hyperparamètres avec GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres trouvés
best_params = grid_search.best_params_
print(f"Meilleurs hyperparamètres : {best_params}")

# Modèle optimisé
rf_best_model = grid_search.best_estimator_

# Étape 4 : Entraînement du modèle optimisé
rf_best_model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = rf_best_model.predict(X_test)
y_pred_proba = rf_best_model.predict_proba(X_test)[:, 1]  # Probabilité pour la classe positive

# Évaluer les performances
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy sur le test : {accuracy:.4f}")
print(f"Rapport de classification :\n{report}")

# Étape 5 : Courbe ROC et AUC
if len(np.unique(y)) == 2:  # Vérification si problème binaire
    auc_score = roc_auc_score(y_test, y_pred_proba)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    print(f"AUC-ROC Score : {auc_score:.4f}")

    # Tracer la courbe ROC
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc_score:.4f})')
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')
    plt.xlabel('Taux de faux positifs (FPR)')
    plt.ylabel('Taux de vrais positifs (TPR)')
    plt.title('Courbe ROC')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

# Étape 6 : Importance des caractéristiques
feature_importances = rf_best_model.feature_importances_
sorted_importances = sorted(zip(important_columns, feature_importances), key=lambda x: x[1], reverse=True)

print("Importance des caractéristiques :")
for feature, importance in sorted_importances:
    print(f"{feature}: {importance:.4f}")

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Supposons que 'SVHC' soit la variable cible
X = df_train.drop(columns=['SVHC'])
y = df_train['SVHC']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer le modèle de Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
rf_model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = rf_model.predict(X_test)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")